In [1]:
import numpy as np, os
import pandas as pd

# import optuna

# https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/274717 
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

import warnings
warnings.filterwarnings(action='ignore')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

print(tf.__version__)
print(keras.__version__)

In [3]:
import random    
seed_num = 48
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 4069), (6182,), (1545, 10, 4069), (1545,))

In [4]:
COLS = list(pd.read_csv('/project/guri/data/data2d_forposter.csv').columns)
# COLS = COLS[:100]

In [5]:
len(set(COLS))

4069

In [6]:
seed_num = 48
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))
#fit
ep = 500
pa = 30
early_stop = EarlyStopping(monitor='val_acc', patience=pa, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= "adam", loss = "binary_crossentropy", metrics=['acc'])

In [12]:
checkpoint_filepath = f"test.hdf5"

lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
es = EarlyStopping(monitor="val_loss", patience=5, verbose=1, mode="min", restore_best_weights=True)
sv = keras.callbacks.ModelCheckpoint(
    checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
)
        
history = lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=ep, callbacks=[lr, es, sv], shuffle=False)

# results = []
# print(' Computing LSTM feature importance...')

# # COMPUTE BASELINE (NO SHUFFLE)
# oof_preds = lstm.predict(X_test, verbose=0)
# baseline_mae = np.mean(np.abs( oof_preds-y_test ))
# results.append({'feature':'BASELINE','baseline_mae':baseline_mae}) 

Epoch 1/500
37/37 [==============================] - 3s 87ms/step - loss: 0.6761 - acc: 0.6001 - val_loss: 0.6698 - val_acc: 0.6074

Epoch 00001: val_loss improved from inf to 0.66979, saving model to test.hdf5
Epoch 2/500
37/37 [==============================] - 3s 71ms/step - loss: 0.6689 - acc: 0.6068 - val_loss: 0.6536 - val_acc: 0.6074

Epoch 00002: val_loss improved from 0.66979 to 0.65364, saving model to test.hdf5
Epoch 3/500
37/37 [==============================] - 3s 71ms/step - loss: 0.5930 - acc: 0.6760 - val_loss: 0.5628 - val_acc: 0.7173

Epoch 00003: val_loss improved from 0.65364 to 0.56284, saving model to test.hdf5
Epoch 4/500
37/37 [==============================] - 3s 70ms/step - loss: 0.4734 - acc: 0.7877 - val_loss: 0.5243 - val_acc: 0.7542

Epoch 00004: val_loss improved from 0.56284 to 0.52430, saving model to test.hdf5
Epoch 5/500
37/37 [==============================] - 3s 71ms/step - loss: 0.4058 - acc: 0.8279 - val_loss: 0.5374 - val_acc: 0.7471

Epoch 00005

`tf.distribute.get_strategy()`
- 훈련을 여러 처리 장치들로 분산시키는 것
- gpu 쓰는 것 X

In [16]:
gpu_strategy = tf.distribute.get_strategy()

with gpu_strategy.scope():
    for k in tqdm(range(len(COLS))):

        # SHUFFLE FEATURE K
        save_col = X_test[:,:,k].copy()
        np.random.shuffle(X_test[:,:,k])

        # COMPUTE OOF MAE WITH FEATURE K SHUFFLED
        oof_preds = lstm.predict(X_test, verbose=0)
        mae = np.mean(np.abs( oof_preds-y_test ))

        results.append({'feature':COLS[k],'mae':mae})
        X_test[:,:,k] = save_col
        # DISPLAY LSTM FEATURE IMPORTANCE
    #     print()
        df = pd.DataFrame(results)
        df = df.sort_values('mae')

  0%|          | 0/4069 [00:00<?, ?it/s]

In [18]:
df.sort_values('mae', ascending=False)

,feature,baseline_mae,mae
3971,63323026201,NaN,0.485214
157,50861,NaN,0.484758
28,50861,NaN,0.484739
3184,51079000220,NaN,0.484708
4,50805,NaN,0.484701
...,...,...,...
43,50882,NaN,0.483335
65,50912,NaN,0.483287
259,51006,NaN,0.483185
130,51006,NaN,0.483179


In [10]:
# def BinaryCrossEntropy(y_true, y_pred):
#     m = y_true.shape[1]
#     y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
#     # Calculating loss
#     loss = -1/m * (np.dot(y_true.T, np.log(y_pred)) + np.dot((1 - y_true).T, np.log(1 - y_pred)))

#     return loss

In [13]:
import time
from sklearn import metrics 
from tensorflow.keras.losses import BinaryCrossentropy
bce = BinaryCrossentropy()

def item_acc(num):
    results = []
    test = X_test.copy()
    save_col = test[:,:,num].copy()
    np.random.shuffle(test[:,:,num])

    preds = lstm.predict(test)
    loss = bce(y_test, preds).numpy()
    
    preds[preds>0.5]=1
    preds[preds<=0.5]=0
    acc = metrics.accuracy_score(y_test, preds)
    
    results.append({'feature':COLS[num],'bce':loss})
    test[:,:,num] = save_col

    print(f'item idx {num} acc & loss :', acc, loss)
    return results

In [16]:
gpu_strategy = tf.distribute.get_strategy()

total_result = []
with gpu_strategy.scope():
    for k in tqdm(range(len(COLS))):
        total_result.append(item_acc(k))

  0%|          | 0/4069 [00:00<?, ?it/s]

item idx 0 acc & loss : 0.7695792880258899 0.9416786
item idx 1 acc & loss : 0.7754045307443366 0.94078207
item idx 2 acc & loss : 0.7747572815533981 0.94061154
item idx 3 acc & loss : 0.7754045307443366 0.94079375
item idx 4 acc & loss : 0.7754045307443366 0.9408754
item idx 5 acc & loss : 0.7747572815533981 0.9408267
item idx 6 acc & loss : 0.7760517799352751 0.9406753
item idx 7 acc & loss : 0.7766990291262136 0.9407451
item idx 8 acc & loss : 0.7773462783171521 0.9402641
item idx 9 acc & loss : 0.7754045307443366 0.94079375
item idx 10 acc & loss : 0.7799352750809061 0.94062287
item idx 11 acc & loss : 0.7754045307443366 0.94067603
item idx 12 acc & loss : 0.7799352750809061 0.9405592
item idx 13 acc & loss : 0.7760517799352751 0.94086486
item idx 14 acc & loss : 0.7754045307443366 0.94072884
item idx 15 acc & loss : 0.7754045307443366 0.94078016
item idx 16 acc & loss : 0.7754045307443366 0.94079155
item idx 17 acc & loss : 0.7754045307443366 0.94079363
item idx 18 acc & loss : 0.

item idx 150 acc & loss : 0.7754045307443366 0.94079363
item idx 151 acc & loss : 0.7754045307443366 0.94079363
item idx 152 acc & loss : 0.7754045307443366 0.9407934
item idx 153 acc & loss : 0.7754045307443366 0.9407933
item idx 154 acc & loss : 0.7741100323624596 0.9407326
item idx 155 acc & loss : 0.7754045307443366 0.94052416
item idx 156 acc & loss : 0.7754045307443366 0.94078666
item idx 157 acc & loss : 0.7754045307443366 0.9406869
item idx 158 acc & loss : 0.7754045307443366 0.94079363
item idx 159 acc & loss : 0.7754045307443366 0.94078374
item idx 160 acc & loss : 0.7754045307443366 0.9408238
item idx 161 acc & loss : 0.7754045307443366 0.94079256
item idx 162 acc & loss : 0.7760517799352751 0.9406847
item idx 163 acc & loss : 0.7754045307443366 0.94079363
item idx 164 acc & loss : 0.7754045307443366 0.94079363
item idx 165 acc & loss : 0.7754045307443366 0.94079363
item idx 166 acc & loss : 0.7754045307443366 0.94079363
item idx 167 acc & loss : 0.7754045307443366 0.9407936

item idx 298 acc & loss : 0.7754045307443366 0.9406653
item idx 299 acc & loss : 0.7754045307443366 0.9408012
item idx 300 acc & loss : 0.7754045307443366 0.94076455
item idx 301 acc & loss : 0.7747572815533981 0.94033533
item idx 302 acc & loss : 0.7754045307443366 0.94080144
item idx 303 acc & loss : 0.7754045307443366 0.94080925
item idx 304 acc & loss : 0.7760517799352751 0.9408025
item idx 305 acc & loss : 0.7773462783171521 0.9400841
item idx 306 acc & loss : 0.7754045307443366 0.9406431
item idx 307 acc & loss : 0.7741100323624596 0.9399378
item idx 308 acc & loss : 0.7754045307443366 0.94080704
item idx 309 acc & loss : 0.7760517799352751 0.94078547
item idx 310 acc & loss : 0.7754045307443366 0.9408162
item idx 311 acc & loss : 0.7754045307443366 0.94079363
item idx 312 acc & loss : 0.7766990291262136 0.94128317
item idx 313 acc & loss : 0.7754045307443366 0.9407942
item idx 314 acc & loss : 0.7754045307443366 0.9407612
item idx 315 acc & loss : 0.7754045307443366 0.94079363
i

item idx 446 acc & loss : 0.7754045307443366 0.94079363
item idx 447 acc & loss : 0.7754045307443366 0.94079363
item idx 448 acc & loss : 0.7754045307443366 0.940793
item idx 449 acc & loss : 0.7754045307443366 0.9407933
item idx 450 acc & loss : 0.7754045307443366 0.94079363
item idx 451 acc & loss : 0.7754045307443366 0.94079363
item idx 452 acc & loss : 0.7754045307443366 0.94079363
item idx 453 acc & loss : 0.7754045307443366 0.94079363
item idx 454 acc & loss : 0.7754045307443366 0.94079363
item idx 455 acc & loss : 0.7754045307443366 0.94079363
item idx 456 acc & loss : 0.7754045307443366 0.94079363
item idx 457 acc & loss : 0.7754045307443366 0.9407981
item idx 458 acc & loss : 0.7754045307443366 0.94079363
item idx 459 acc & loss : 0.7754045307443366 0.9407966
item idx 460 acc & loss : 0.7754045307443366 0.94079316
item idx 461 acc & loss : 0.7754045307443366 0.94079363
item idx 462 acc & loss : 0.7754045307443366 0.94079363
item idx 463 acc & loss : 0.7754045307443366 0.940793

...

item idx 3359 acc & loss : 0.7754045307443366 0.94079363
item idx 3360 acc & loss : 0.7754045307443366 0.94075876
item idx 3361 acc & loss : 0.7754045307443366 0.94079375
item idx 3362 acc & loss : 0.7754045307443366 0.94080293
item idx 3363 acc & loss : 0.7754045307443366 0.94079363
item idx 3364 acc & loss : 0.7754045307443366 0.94079363
item idx 3365 acc & loss : 0.7754045307443366 0.94079363
item idx 3366 acc & loss : 0.7754045307443366 0.94075507
item idx 3367 acc & loss : 0.7754045307443366 0.94081855
item idx 3368 acc & loss : 0.7754045307443366 0.94081074
item idx 3369 acc & loss : 0.7754045307443366 0.94079363
item idx 3370 acc & loss : 0.7754045307443366 0.94079363
item idx 3371 acc & loss : 0.7754045307443366 0.94079363
item idx 3372 acc & loss : 0.7754045307443366 0.94079363
item idx 3373 acc & loss : 0.7747572815533981 0.9407975
item idx 3374 acc & loss : 0.7754045307443366 0.94079363
item idx 3375 acc & loss : 0.7754045307443366 0.94079363
item idx 3376 acc & loss : 0.775

item idx 3504 acc & loss : 0.7754045307443366 0.94079363
item idx 3505 acc & loss : 0.7754045307443366 0.9407887
item idx 3506 acc & loss : 0.7754045307443366 0.94079363
item idx 3507 acc & loss : 0.7747572815533981 0.9406745
item idx 3508 acc & loss : 0.7754045307443366 0.94079363
item idx 3509 acc & loss : 0.7747572815533981 0.94069135
item idx 3510 acc & loss : 0.7754045307443366 0.94079363
item idx 3511 acc & loss : 0.7754045307443366 0.94078356
item idx 3512 acc & loss : 0.7754045307443366 0.9407969
item idx 3513 acc & loss : 0.7754045307443366 0.9408001
item idx 3514 acc & loss : 0.7754045307443366 0.940793
item idx 3515 acc & loss : 0.7754045307443366 0.94079363
item idx 3516 acc & loss : 0.7754045307443366 0.940841
item idx 3517 acc & loss : 0.7754045307443366 0.94079363
item idx 3518 acc & loss : 0.7754045307443366 0.9407813
item idx 3519 acc & loss : 0.7754045307443366 0.94080853
item idx 3520 acc & loss : 0.7754045307443366 0.9408339
item idx 3521 acc & loss : 0.775404530744

item idx 3649 acc & loss : 0.7760517799352751 0.94065154
item idx 3650 acc & loss : 0.7754045307443366 0.9408081
item idx 3651 acc & loss : 0.7754045307443366 0.9407981
item idx 3652 acc & loss : 0.7754045307443366 0.94079345
item idx 3653 acc & loss : 0.7754045307443366 0.94080275
item idx 3654 acc & loss : 0.7754045307443366 0.94079363
item idx 3655 acc & loss : 0.7754045307443366 0.94080746
item idx 3656 acc & loss : 0.7754045307443366 0.9407716
item idx 3657 acc & loss : 0.7754045307443366 0.94079363
item idx 3658 acc & loss : 0.7747572815533981 0.94067353
item idx 3659 acc & loss : 0.7754045307443366 0.94079363
item idx 3660 acc & loss : 0.7754045307443366 0.94079363
item idx 3661 acc & loss : 0.7754045307443366 0.94056857
item idx 3662 acc & loss : 0.7773462783171521 0.9403487
item idx 3663 acc & loss : 0.7754045307443366 0.9407923
item idx 3664 acc & loss : 0.7754045307443366 0.94079363
item idx 3665 acc & loss : 0.7754045307443366 0.94079363
item idx 3666 acc & loss : 0.7754045

item idx 3794 acc & loss : 0.7754045307443366 0.94079363
item idx 3795 acc & loss : 0.7754045307443366 0.94079363
item idx 3796 acc & loss : 0.7754045307443366 0.94079363
item idx 3797 acc & loss : 0.7754045307443366 0.94079363
item idx 3798 acc & loss : 0.7754045307443366 0.94079363
item idx 3799 acc & loss : 0.7754045307443366 0.940756
item idx 3800 acc & loss : 0.7754045307443366 0.94079363
item idx 3801 acc & loss : 0.7747572815533981 0.9408353
item idx 3802 acc & loss : 0.7754045307443366 0.94079363
item idx 3803 acc & loss : 0.7754045307443366 0.94079363
item idx 3804 acc & loss : 0.7754045307443366 0.94079244
item idx 3805 acc & loss : 0.7754045307443366 0.94079363
item idx 3806 acc & loss : 0.7754045307443366 0.94079363
item idx 3807 acc & loss : 0.7754045307443366 0.9406057
item idx 3808 acc & loss : 0.7754045307443366 0.94079953
item idx 3809 acc & loss : 0.7754045307443366 0.9407917
item idx 3810 acc & loss : 0.7754045307443366 0.9407937
item idx 3811 acc & loss : 0.77540453

item idx 3939 acc & loss : 0.7754045307443366 0.94080484
item idx 3940 acc & loss : 0.7754045307443366 0.94079363
item idx 3941 acc & loss : 0.7754045307443366 0.94079363
item idx 3942 acc & loss : 0.7754045307443366 0.9407388
item idx 3943 acc & loss : 0.7754045307443366 0.94079363
item idx 3944 acc & loss : 0.7754045307443366 0.940817
item idx 3945 acc & loss : 0.7747572815533981 0.94084734
item idx 3946 acc & loss : 0.7747572815533981 0.94083744
item idx 3947 acc & loss : 0.7754045307443366 0.94079363
item idx 3948 acc & loss : 0.7754045307443366 0.94080746
item idx 3949 acc & loss : 0.7760517799352751 0.94091195
item idx 3950 acc & loss : 0.7754045307443366 0.94079363
item idx 3951 acc & loss : 0.7754045307443366 0.94079363
item idx 3952 acc & loss : 0.7754045307443366 0.9407944
item idx 3953 acc & loss : 0.7754045307443366 0.94079363
item idx 3954 acc & loss : 0.7754045307443366 0.94079363
item idx 3955 acc & loss : 0.7754045307443366 0.94079363
item idx 3956 acc & loss : 0.775404

In [18]:
total_result

[[{'feature': '0', 'bce': 0.9416786}],
 [{'feature': '50803', 'bce': 0.94078207}],
 [{'feature': '50804', 'bce': 0.94061154}],
 [{'feature': '50805', 'bce': 0.94079375}],
 [{'feature': '50806', 'bce': 0.9408754}],
 [{'feature': '50808', 'bce': 0.9408267}],
 [{'feature': '50809', 'bce': 0.9406753}],
 [{'feature': '50811', 'bce': 0.9407451}],
 [{'feature': '50813', 'bce': 0.9402641}],
 [{'feature': '50814', 'bce': 0.94079375}],
 [{'feature': '50818', 'bce': 0.94062287}],
 [{'feature': '50820', 'bce': 0.94067603}],
 [{'feature': '50821', 'bce': 0.9405592}],
 [{'feature': '50822', 'bce': 0.94086486}],
 [{'feature': '50824', 'bce': 0.94072884}],
 [{'feature': '50852', 'bce': 0.94078016}],
 [{'feature': '50853', 'bce': 0.94079155}],
 [{'feature': '50856', 'bce': 0.94079363}],
 [{'feature': '50858', 'bce': 0.94079363}],
 [{'feature': '50861', 'bce': 0.9408003}],
 [{'feature': '50862', 'bce': 0.9412688}],
 [{'feature': '50863', 'bce': 0.9408634}],
 [{'feature': '50864', 'bce': 0.94079363}],
 [